## 16단계: 복잡한 계산 그래프(구현 편)

> 이번 단계에서는 15단계에서 설명한 이론을 구현합니다. \
가장 먼저 '세대'를 설정하고, 역전파 시 최근 세대의 함수부터 꺼내도록 합니다. \
그러면 아무리 계산 그래프가 복잡해도 올바른 순서로 역전파가 이루어집니다.

### 16.1 세대 추가

Variable 클래스에 세대(generation)을 추가하자. \
이 때, Variable의 세대는 부모 함수의 세대보다 1만큼 더 크게 설정한다.

<img src="images/그림 16-1.png" width=400/>

In [3]:
import numpy as np

class Variable:
    def __init__(self, data):
        if not isinstance(data, (type(None), np.ndarray)):
            raise TypeError(f'{type(data)}은(는) 지원하지 않습니다.')
        
        self.data = data
        self.grad = None
        self.creator = None
        self.generation = 0  # 세대 수를 기록하는 변수수

    def set_creator(self, func):
        self.creator = func
        self.generation = func.generation + 1  # 세대 기록(부모 세대 + 1)
    
    def backward(self):
        if self.grad is None:
            self.grad = np.ones_like(self.data)
        
        funcs = [self.creator]
        while funcs:
            f = funcs.pop()
            gys = [output.grad for output in f.outputs]
            gxs = f.backward(*gys)
            if not isinstance(gxs, tuple):
                gxs = (gxs,)
            
            for x, gx in zip(f.inputs, gxs):
                if x.grad is None:
                    x.grad = gx
                else:
                    x.grad = x.grad + gx
                
                if x.creator is not None:
                    funcs.append(x.creator)
    
    def cleargrad(self):
        self.grad = None

Function의 generation은 입력 변수와 같은 값으로 설정한다. \
만약 입력 변수가 둘 이상이라면 가장 큰 generation 값을 선택한다.

<img src="images/그림 16-2.png" width=550/>

In [4]:
def as_array(x):
    if np.isscalar(x):
        return np.asarray(x)
    return x


class Function:
    def __call__(self, *inputs):
        xs = [x.data for x in inputs]
        ys = self.forward(*xs)
        if not isinstance(ys, tuple):
            ys = (ys,)
        outputs = [Variable(as_array(y)) for y in ys]
        
        self.generation = max([x.generation for x in inputs])  # 입력 변수 세대 중 가장 큰 세대 선택택
        for output in outputs:
            output.set_creator(self)
        self.inputs = inputs
        self.outputs = outputs
        
        return outputs if len(outputs) > 1 else outputs[0]
    
    def forward(self, xs):
        raise NotImplementedError()
    
    def backward(self, gys):
        raise NotImplementedError()

### 16.2 세대 순으로 꺼내기

지금까지 수정한 내용을 반영하면 다음과 같이 세대가 설정된다.

<img src="images/그림 16-3.png" width=550/>

이제 역전파 시 세대가 큰 순으로 함수를 꺼내기만 하면 된다.

### 16.3 Variable 클래스의 backward

위 내용을 구현해보자.

In [ ]:
class Variable:
    def __init__(self, data):
        if not isinstance(data, (type(None), np.ndarray)):
            raise TypeError(f'{type(data)}은(는) 지원하지 않습니다.')
        
        self.data = data
        self.grad = None
        self.creator = None
        self.generation = 0  # 세대 수를 기록하는 변수수

    def set_creator(self, func):
        self.creator = func
        self.generation = func.generation + 1  # 세대 기록(부모 세대 + 1)
    
    def backward(self):
        if self.grad is None:
            self.grad = np.ones_like(self.data)
        
        # 수정 코드
        funcs = []
        seen_set = set()
        
        def add_func(f):
            if f not in seen_set:
                funcs.append(f)
                seen_set.add(f)
                funcs.sort(key=lambda x: x.generation)
        
        add_func(self.creator)
        ########################
        
        # funcs = [self.creator]
        while funcs:
            f = funcs.pop()
            gys = [output.grad for output in f.outputs]
            gxs = f.backward(*gys)
            if not isinstance(gxs, tuple):
                gxs = (gxs,)
            
            for x, gx in zip(f.inputs, gxs):
                if x.grad is None:
                    x.grad = gx
                else:
                    x.grad = x.grad + gx
                
                if x.creator is not None:
                    # 기존 코드
                    # funcs.append(x.creator)
                    
                    # 수정 코드
                    add_func(x.creator)
    
    def cleargrad(self):
        self.grad = None

`Variable` 클래스의 `backward` 메서드를 수정하여 구현했다. \
함수를 `funcs` 리스트에 추가할 때마다 세대 순으로 정렬하도록 하며, 이미 추가된 함수는 `seen_set`으로 걸러준다. \
하지만 매번 `funcs`를 세대 순으로 정렬하는 것은 비효율적이기 때문에, '우선순위 큐'를 이용하면 더 좋다. \
아래는 우선순위 큐를 이용한 구현이다.

In [34]:
from heapq import heappush, heappop


class Function:
    def __call__(self, *inputs):
        xs = [x.data for x in inputs]
        ys = self.forward(*xs)
        if not isinstance(ys, tuple):
            ys = (ys,)
        outputs = [Variable(as_array(y)) for y in ys]
        
        self.generation = max([x.generation for x in inputs])  # 입력 변수 세대 중 가장 큰 세대 선택택
        for output in outputs:
            output.set_creator(self)
        self.inputs = inputs
        self.outputs = outputs
        
        return outputs if len(outputs) > 1 else outputs[0]

    # 함수 간 순서 비교를 위한 __lt__ 메서드 구현
    def __lt__(self, other):
        # 세대가 크면 더 작다고 정의한다.
        return self.generation > other.generation
    
    def forward(self, xs):
        raise NotImplementedError()
    
    def backward(self, gys):
        raise NotImplementedError()



class Variable:
    def __init__(self, data):
        if not isinstance(data, (type(None), np.ndarray)):
            raise TypeError(f'{type(data)}은(는) 지원하지 않습니다.')
        
        self.data = data
        self.grad = None
        self.creator = None
        self.generation = 0  # 세대 수를 기록하는 변수수

    def set_creator(self, func):
        self.creator = func
        self.generation = func.generation + 1  # 세대 기록(부모 세대 + 1)
    
    def backward(self):
        if self.grad is None:
            self.grad = np.ones_like(self.data)
        
        # 수정 코드 (ver. 우선순위 큐)
        funcs_heap = []
        seen_set = set()
        
        def add_func(f):
            if f not in seen_set:
                # 함수를 우선순위 큐에 추가
                heappush(funcs_heap, f)
                seen_set.add(f)
                # 정렬하는 부분 제거함
        
        add_func(self.creator)
        ########################
        
        while funcs_heap:
            # 세대가 가장 큰 함수를 꺼낸다.
            f = heappop(funcs_heap)
            gys = [output.grad for output in f.outputs]
            gxs = f.backward(*gys)
            if not isinstance(gxs, tuple):
                gxs = (gxs,)
            
            for x, gx in zip(f.inputs, gxs):
                if x.grad is None:
                    x.grad = gx
                else:
                    x.grad = x.grad + gx
                
                if x.creator is not None:
                    # 기존 코드
                    # funcs.append(x.creator)
                    
                    # 수정 코드
                    add_func(x.creator)
    
    def cleargrad(self):
        self.grad = None

### 16.4 동작 확인

시험 삼아 `그림 16-4`의 계산을 미분해보자.

<img src="images/그림 16-4.png" width=600/>

In [35]:
# 필요 모듈 정의

class Add(Function):
    def forward(self, x0, x1):
        y = x0 + x1
        return y
    
    def backward(self, gy):
        return gy, gy


class Square(Function):
    def forward(self, x):
        y = x ** 2
        return y
    
    def backward(self, gy):
        x = self.inputs[0].data
        gx = 2 * x * gy
        return gx


def add(x0, x1):
    return Add()(x0, x1)

def square(x):
    return Square()(x)

In [36]:
x = Variable(np.array(2.0))
a = square(x)
y = add(square(a), square(a))
y.backward()

print(y.data)
print(x.grad)  # 예상값: 64.0

32.0
64.0


이제 DeZero는 복잡한 연결도 제대로 역전파 미분을 수행할 수 있게 되었다. \
가령, 구현만 한다면 다음과 같은 계산 그래프도 미분할 수 있을 것이다.

<img src="images/그림 16-5.png" width=600/>